In [27]:
import os
from pathlib import Path

In [28]:
import mip

ModuleNotFoundError: No module named 'mip'

## using julia's image as example

In [26]:
sample_id = '20260818_BONE_IRD_Slide_1'
qptiff_path = Path('/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/analysis/CODEX/Run_08182026/BONE_IRD_Slide_1_Scan1.qptiff')
assert qptiff_path.is_file()

In [2]:
qptiff_path = Path('/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/analysis/CODEX/Run_08182026/BONE_IRD_Slide_1_Scan1.qptiff')
assert qptiff_path.is_file()

In [42]:
bbox = (10000,12000,10000,11000)
pixel_type = 'uint8'
subresolutions = 4
n_channels = 5

tf = tifffile.TiffFile(qptiff_path)
s = tf.series[0] # full res tiffs are in the first series
# n_channels = s.get_shape()[0]
shape = s.pages[0].shape

f'{n_channels} total markers', f'writing as {pixel_type}'

('5 total markers', 'writing as uint8')

In [43]:
if bbox is None:
    data = np.zeros((shape[1], shape[0], n_channels, 1, 1),
                    dtype=np.uint8 if pixel_type=='uint8' else np.uint16)
else:
    data = np.zeros((bbox[3] - bbox[2], bbox[1] - bbox[0], n_channels, 1, 1),
                    dtype=np.uint8 if pixel_type=='uint8' else np.uint16)
data.shape

(1000, 2000, 5, 1, 1)

In [44]:
biomarkers = []
for i, p in enumerate(s.pages[:n_channels]):
    img = p.asarray()
    d = tifffile.xml2dict(p.description)['PerkinElmer-QPI-ImageDescription']
    biomarker = d['Biomarker']
    print(biomarker)
    biomarkers.append(biomarker)
    if bbox is not None:
        r1, r2, c1, c2 = bbox
        img = img[r1:r2, c1:c2]

    if pixel_type=='uint8':
        img = img.astype(np.uint8)
    else:
        img = img.astype(np.float32)
        img /= img.max()
        img *= 65535
        img = img.astype(np.uint16)
        logging.info('writing as uint16')

    data[..., i, 0, 0] = np.swapaxes(img, 0, 1)

DAPI
CD8
CD20
CD45
TACI


In [45]:
data.shape

(1000, 2000, 5, 1, 1)

In [46]:
PHENOCYCLER_PIXELS_PER_MICRON = 1.9604911906033102
from ome_types import from_tiff, from_xml, to_xml, model

o = model.OME()
o.images.append(
    model.Image(
        id='Image:0',
        pixels=model.Pixels(
            dimension_order='XYCZT',
            size_c=n_channels,
            size_t=1,
            size_x=data.shape[0],
            size_y=data.shape[1],
            size_z=data.shape[3],
            type=pixel_type,
            big_endian=False,
            channels=[model.Channel(id=f'Channel:{i}', name=c) for i, c in enumerate(biomarkers)],
            physical_size_x=1 / PHENOCYCLER_PIXELS_PER_MICRON,
            physical_size_y=1 / PHENOCYCLER_PIXELS_PER_MICRON,
            physical_size_x_unit='µm',
            physical_size_y_unit='µm'
        )
    )
)

im = o.images[0]
for i in range(len(im.pixels.channels)):
    im.pixels.planes.append(model.Plane(the_c=i, the_t=0, the_z=0))
im.pixels.tiff_data_blocks.append(model.TiffData(plane_count=len(im.pixels.channels)))

o

OME(
   images=[<1 field_type>],
   structured_annotations={'xml_annotations': [], 'file_annotations': [], 'list_annotations': [], 'long_annotations': [], 'double_annotations': [], 'comment_annotations': [], 'boolean_annotations': [], 'timestamp_annotations': [], 'tag_annotations': [], 'term_annotations': [], 'map_annotations': []},
)

In [49]:
# 'x y c z t -> t c y x z'

def write_HTAN_ome(output_fp, data, ome_model, subresolutions=4):
    import gc
    with tifffile.TiffWriter(output_fp, ome=True, bigtiff=True) as out_tif:
        opts = {
            'compression': 'LZW',
        }
        out_tif.write( 
            rearrange(data[..., 0, 0], 'x y c -> c y x'),
            subifds=subresolutions,
            **opts
        )
        gc.collect()
        for level in range(subresolutions):
            mag = 2**(level + 1)
            print(f'writting subres {mag}')
            # x = data[::mag, ::mag, :, 0, 0]
            # x = rearrange(torch.tensor(data[..., 0, 0]), 'w h c -> c h w')
            
            # shape = (int(x.shape[-2] / mag), int(x.shape[-1] / mag))
            # sampled = rearrange(
            #     TF.resize(x, shape, antialias=True),
            #     'c h w -> w h c 1 1'
            # )
            # del(x)
            # gc.collect()
            out_tif.write(
                rearrange(data[::mag, ::mag, :, 0, 0], 'x y c -> c y x'),
                subfiletype=1,
                **opts
            )
            # del(sampled)
        xml_str = to_xml(ome_model)
        out_tif.overwrite_description(xml_str.encode())

In [50]:
write_HTAN_ome('../../../sandbox/temp.ome.tif', data, o, subresolutions=4)

writting subres 2
writting subres 4
writting subres 8
writting subres 16


In [34]:
Path('../../../sandbox/temp.ome.tif').absolute()

PosixPath('/diskmnt/Users2/estorrs/imaging-analysis/notebooks/multiplex_analysis_examples/../../../sandbox/temp.ome.tif')

In [3]:
def generate_ome_from_qptiff(qptiff_fp, output_fp, bbox=None, pixel_type='uint8', subresolutions=4):
    """
    Generate an HTAN compatible ome tiff from qptiff output by codex phenocycler
    """
    tf = tifffile.TiffFile(qptiff_fp)
    s = tf.series[0] # full res tiffs are in the first series
    n_channels = s.get_shape()[0]
    shape = s.pages[0].shape

    logging.info(f'{n_channels} total markers')
    logging.info(f'writing as {pixel_type}')

    if bbox is None:
        data = np.zeros((shape[1], shape[0], n_channels, 1, 1),
                        dtype=np.uint8 if pixel_type=='uint8' else np.uint16)
    else:
        logging.info(f'bbox detected: {bbox}')
        data = np.zeros((bbox[3] - bbox[2], bbox[1] - bbox[0], n_channels, 1, 1),
                        dtype=np.uint8 if pixel_type=='uint8' else np.uint16)
    biomarkers = []
    for i, p in enumerate(s.pages):
        img = p.asarray()
        d = tifffile.xml2dict(p.description)['PerkinElmer-QPI-ImageDescription']
        biomarker = d['Biomarker']
        logging.info(f'loading {biomarker}')
        biomarkers.append(biomarker)
        if bbox is not None:
            r1, r2, c1, c2 = bbox
            img = img[r1:r2, c1:c2]

        if pixel_type=='uint8':
            img = img.astype(np.uint8)
        else:
            img = img.astype(np.float32)
            img /= img.max()
            img *= 65535
            img = img.astype(np.uint16)
            logging.info('writing as uint16')

        data[..., i, 0, 0] = np.swapaxes(img, 0, 1)

    o = model.OME()
    o.images.append(
        model.Image(
            id='Image:0',
            pixels=model.Pixels(
                dimension_order='XYCZT',
                size_c=n_channels,
                size_t=1,
                size_x=data.shape[0],
                size_y=data.shape[1],
                size_z=data.shape[3],
                type=pixel_type,
                big_endian=False,
                channels=[model.Channel(id=f'Channel:{i}', name=c) for i, c in enumerate(biomarkers)],
                physical_size_x=1 / PHENOCYCLER_PIXELS_PER_MICRON,
                physical_size_y=1 / PHENOCYCLER_PIXELS_PER_MICRON,
                physical_size_x_unit='µm',
                physical_size_y_unit='µm'
            )
        )
    )

    im = o.images[0]
    for i in range(len(im.pixels.channels)):
        im.pixels.planes.append(model.Plane(the_c=i, the_t=0, the_z=0))
    im.pixels.tiff_data_blocks.append(model.TiffData(plane_count=len(im.pixels.channels)))

    write_HTAN_ome(output_fp, data, o, subresolutions=subresolutions)

In [ ]:
def write_HTAN_ome(output_fp, data, ome_model, subresolutions=4):
    import gc
    with tifffile.TiffWriter(output_fp, ome=True, bigtiff=True) as out_tif:
        opts = {
            'compression': 'LZW',
        }
        logging.info(f'writting full res image')
        out_tif.write(
            rearrange(data, 'x y c z t -> t c y x z'),
            subifds=subresolutions,
            **opts
        )
        gc.collect()
        for level in range(subresolutions):
            mag = 2**(level + 1)
            logging.info(f'writting subres {mag}')
            x = rearrange(torch.tensor(data[..., 0, 0]), 'w h c -> c h w')
            shape = (int(x.shape[-2] / mag), int(x.shape[-1] / mag))
            sampled = rearrange(
                TF.resize(x, shape, antialias=True),
                'c h w -> w h c 1 1'
            )
            logging.info(f'subresolution shape: {shape}')
            del(x)
            gc.collect()
            out_tif.write(
                rearrange(sampled.numpy().astype(np.uint8), 'x y c z t -> t c y x z'),
                subfiletype=1,
                **opts
            )
            del(sampled)
        xml_str = to_xml(ome_model)
        out_tif.overwrite_description(xml_str.encode())